In [3]:
import pathlib
import os
from keras.layers import Dense, Dropout, Activation, LSTM, Bidirectional, Embedding
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku
from keras.utils import pad_sequences
import tensorflow as tf
from numpy.random import seed
import pandas as pd
import numpy as np
# simplified chinese tokenizer
import jieba
import time
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
import re
import nltk

seed(1)
tf.random.set_seed(2)
punc = "！？｡。＂＃＄％＆＇（）＊＋，－／：；＜＝＞＠［＼］＾＿｀｛｜｝～｟｠｢｣､、〃》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〾〿–—‘’‛“”„‟…‧﹏.《》（）+-=()""''/="

# skipped directories
# SKIP = ["0", "a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z"]
SKIP = ["a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "r", "s", "t", "u", "v",
        "w", "x", "y", "z"]
# SKIP = []

t1 = time.time()


def get_all_items(root: pathlib.Path, exclude):
    itemList = []
    for item in root.iterdir():
        if item.name in exclude:
            continue
        if item.is_dir():
            itemList.append(get_all_items(item, []))
            continue
        itemList.append(item)
    return itemList

# q = 0
# trainDir = "./Books/Train/"
# for filename in os.listdir(trainDir):
#     fn = trainDir + str(q) + ".txt"
#     originalName = trainDir + filename
#     os.rename(originalName,fn)
#     q += 1


# begin preprocessing
largeDir = pathlib.Path("./Books")
BookList = get_all_items(largeDir, SKIP)
BookList = [item for sublist in BookList for item in sublist]

# clean the dataset
# for path in BookList:
#     print(path)
#     file = open(path, 'r')
#     try:
#         fileStr = file.read()
#     except UnicodeDecodeError as error:
#         file.close()
#         os.remove(path)
#     continue

bigString = ""

for path in BookList:
    with open(path, 'r') as fiction:
        bigString += fiction.read()

# methods to strip punctuation and symbols
# bigString = re.sub(r"[%s]+" %punc, "", bigString)
bigString = re.sub(r'[^\w\s]', '', bigString)

# list of the words in their original order
allTokens = jieba.lcut(bigString, cut_all=False)


print("Corpus length in words: ", len(allTokens))

# os.remove('vocab.txt')

minFreq = 400
wordFreq = {}
for token in allTokens:
    wordFreq[token] = wordFreq.get(token, 0) + 1

skipWords = set()
for k, v in wordFreq.items():
    if wordFreq[k] < minFreq:
        skipWords.add(k)


words = set(allTokens)
print("Unique words before filter: ", len(words))
print("To reduce vocab size, neglect words with appearances < ", minFreq)
words = sorted(set(words) - skipWords)
print("Unique words after filter: ", len(words))
t2 = time.time()

print("Runtime for this cell in seconds: ", t2 - t1)


Corpus length in words:  63168418
Unique words before filter:  450809
To reduce vocab size, neglect words with appearances <  400
Unique words after filter:  11655
Runtime for this cell in seconds:  518.8514316082001


In [4]:
import codecs

words_file_path = "vocab.txt"

words_file = codecs.open(words_file_path, 'w', encoding='gbk')
# hugeStr = "".join(str(words))
# words_file.write(hugeStr)

for w in words:
    if w != "\n":
        words_file.write(w)
        words_file.write("\n")
    else:
        words_file.write(str(w))
words_file.close()



　
一
一一
一丁点
一万
一万年
一下
一下子
一不小心
一世
一丝
一丝一毫
一丝丝
一两个
一个
一个个
一个半
一个多
一个多月
一个月
一串
一举
一举一动
一事
一二
一些
一亮
一人
一代
一件
一任
一份
一众
一伙
一会
一会儿
一位
一体
一侧
一倍
一元
一共
一具
一凛
一出
一击
一刀
一分
一分钟
一切
一切都是
一到
一刹那
一刻
一刻钟
一剑
一副
一动
一动不动
一匹
一千
一半
一双
一双眼
一发
一变
一口
一口气
一句
一只
一台
一号
一叹
一同
一名
一向
一听
一员
一周
一味
一命
一品
一喜
一回
一团
一圈
一地
一场
一块
一块块
一堆
一声
一声令下
一声声
一处
一夜
一夜之间
一大
一大半
一大堆
一大群
一天
一头
一头雾水
一套
一如
一如既往
一定
一家
一家人
一对
一尊
一小
一层
一层层
一届
一左一右
一巴掌
一带
一帮
一幅
一幕
一年
一并
一度
一座
一座座
一张
一心
一惊
一想
一战
一扇
一手
一扫
一批
一把
一抖
一抹
一拍
一招
一拳
一指
一挑
一挥
一掌
一排
一支
一方
一方面
一旁
一族
一无所知
一日
一旦
一早
一时
一时之间
一时半会
一时间
一是
一晃
一本
一朵
一条
一条条
一来
一杯
一枚
一枪
一架
一柄
一栋
一株
一样
一根
一棵
一模一样
一次
一次性
一次次
一款
一步
一步步
一死
一段
一段时间
一段距离
一沉
一波
一派
一流
一清二楚
一滴
一炉
一点
一点一点
一点儿
一点点
一片
一片片
一瓶
一生
一男
一番
一番话
一百
一百万
一百个
一百多
一直
一看
一眼
一瞬
一瞬间
一礼
一种
一秒
一空
一章
一笑
一笔
一筹
一箭
一米
一类
一粒
一系列
一红
一级
一线
一组
一缕
一缩
一群
一翻
一股
一脚
一脸
一致
一般
一般来说
一艘
一行
一见
一角
一言不发
一记
一说
一起
一趟
一跃
一路
一路上
一跳
一身
一转
一轮
一辆
一辈
一辈子
一边
一连
一连串
一遍
一道
一道道
一部
一部分
一重
一门
一闪
一闪而过
一间
一队
一阵
一阵子
一阵阵
一阶
一震
一面
一顶
一项
一顿
一颗
一颤
一首
一黑
丁
七
七七
七个
七八个
七十二
七品
七天
七彩
七星
七杀
七转
七重
万
万一
万万